<a href="https://colab.research.google.com/github/Ludmilaalc/ProjetoIAS/blob/main/cats_dogs_xp_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, zipfile, tensorflow as tf

url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_path = tf.keras.utils.get_file(
    fname="cats_and_dogs_filtered.zip",
    origin=url,
    extract=False,
)

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall("/content")

base_dir = "/content/cats_and_dogs_filtered"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

train_cats_dir = os.path.join(train_dir, "cats")
train_dogs_dir = os.path.join(train_dir, "dogs")
validation_cats_dir = os.path.join(validation_dir, "cats")
validation_dogs_dir = os.path.join(validation_dir, "dogs")

for p in [train_cats_dir, train_dogs_dir, validation_cats_dir, validation_dogs_dir]:
    print(p, "->", len(os.listdir(p)), "imagens")


68606236/68606236 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
/content/cats_and_dogs_filtered/train/cats -> 1000 imagens
/content/cats_and_dogs_filtered/train/dogs -> 1000 imagens
/content/cats_and_dogs_filtered/validation/cats -> 500 imagens
/content/cats_and_dogs_filtered/validation/dogs -> 500 imagens


In [3]:

import tensorflow as tf, zipfile, os
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
zip_path = tf.keras.utils.get_file("cats_and_dogs_filtered.zip", origin=url, extract=False)
with zipfile.ZipFile(zip_path, "r") as z: z.extractall("/content")

from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

def count_trainable_params(model):
    import tensorflow as tf
    return int(sum(tf.size(w).numpy() for w in model.trainable_weights))

INPUT_SHAPE = (128, 220, 3)
base = VGG16(include_top=False, weights=None, input_shape=INPUT_SHAPE)

# Cabeça tipo VGG: Flatten
x = layers.Flatten(name="flatten")(base.output)
x = layers.Dense(4096, activation="relu", name="fc1")(x)
x = layers.Dense(4096, activation="relu", name="fc2")(x)
out = layers.Dense(2, activation="softmax", name="predictions")(x)

model = models.Model(inputs=base.input, outputs=out, name="vgg16_xp_style")
print("Parâmetros treináveis:", f"{count_trainable_params(model):,}".replace(",", "."))


Parâmetros treináveis: 81.839.938


In [4]:

# Parâmetros treináveis da VGG19
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers, models

def count_trainable_params(model):
    import tensorflow as tf
    return int(sum(tf.size(w).numpy() for w in model.trainable_weights))

INPUT_SHAPE = (128, 220, 3)

# Base convolucional
base = VGG19(include_top=False, weights=None, input_shape=INPUT_SHAPE)

x = layers.Flatten(name="flatten")(base.output)
x = layers.Dense(4096, activation="relu", name="fc1")(x)
x = layers.Dense(4096, activation="relu", name="fc2")(x)
out = layers.Dense(2, activation="softmax", name="predictions")(x)

model = models.Model(inputs=base.input, outputs=out, name="vgg19_xp_style")

print("Parâmetros treináveis (VGG19 padrão XP):",
      f"{count_trainable_params(model):,}".replace(",", "."))


Parâmetros treináveis (VGG19 padrão XP): 87.149.634


In [6]:
#ResNet152

import tensorflow as tf, zipfile
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import layers, models


def count_trainable_params(model):
    return int(sum(tf.size(w).numpy() for w in model.trainable_weights))

INPUT_SHAPE = (128, 220, 3)
base = ResNet152(include_top=False, weights=None, input_shape=INPUT_SHAPE)

x = layers.GlobalAveragePooling2D(name="gap")(base.output)
out = layers.Dense(2, activation="softmax", name="predictions")(x)

model = models.Model(base.input, out, name="resnet152_gap_head")
print("Treináveis (ResNet152 + GAP→Dense(2)):",
      f"{count_trainable_params(model):,}".replace(",", "."))


Treináveis (ResNet152 + GAP→Dense(2)): 58.223.618


In [9]:

import os, glob, cv2, numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split


base_dir = "/content/cats_and_dogs_filtered"

train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
train_cats_dir = os.path.join(train_dir, "cats")
train_dogs_dir = os.path.join(train_dir, "dogs")
validation_cats_dir = os.path.join(validation_dir, "cats")
validation_dogs_dir = os.path.join(validation_dir, "dogs")

# Carregar e redimensionar (220, 128)
IMG_W, IMG_H = 220, 128

def load_dir(path, label):
    X, y = [], []
    for fp in sorted(glob.glob(os.path.join(path, "*"))):
        img = cv2.imread(fp)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_W, IMG_H))
        X.append(img)
        y.append(label)
    return X, y

Xc_tr, yc_tr = load_dir(train_cats_dir, 0)
Xd_tr, yd_tr = load_dir(train_dogs_dir, 1)
Xc_va, yc_va = load_dir(validation_cats_dir, 0)
Xd_va, yd_va = load_dir(validation_dogs_dir, 1)

X = np.array(Xc_tr + Xd_tr + Xc_va + Xd_va, dtype=np.uint8)
y = np.array(yc_tr + yd_tr + yc_va + yd_va, dtype=np.int32)

datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="reflect"
)

Xf = X.astype("float32") / 255.0

augmented, labels_aug = [], []
generated = 0
for xb, yb in datagen.flow(Xf, y, batch_size=64, shuffle=True):
    augmented.append(xb)
    labels_aug.append(yb)
    generated += len(xb)
    if generated >= 1000:
        break

X_aug = np.concatenate(augmented, axis=0)[:1000]
y_aug = np.concatenate(labels_aug, axis=0)[:1000]

X_all = np.concatenate([Xf, X_aug], axis=0)
y_all = np.concatenate([y, y_aug], axis=0)


X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.30, random_state=42, stratify=y_all
)

print("X_train.shape:", X_train.shape)


X_train.shape: (2800, 128, 220, 3)


In [10]:

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2, glob, os

base_dir = "/content/cats_and_dogs_filtered"
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")
train_cats_dir = os.path.join(train_dir, "cats")
train_dogs_dir = os.path.join(train_dir, "dogs")
validation_cats_dir = os.path.join(validation_dir, "cats")
validation_dogs_dir = os.path.join(validation_dir, "dogs")


IMG_W, IMG_H = 220, 128

def load_dir(path, label):
    X, y = [], []
    for fp in sorted(glob.glob(os.path.join(path, "*"))):
        img = cv2.imread(fp)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_W, IMG_H))
        X.append(img)
        y.append(label)
    return X, y

Xc_tr, yc_tr = load_dir(train_cats_dir, 0)
Xd_tr, yd_tr = load_dir(train_dogs_dir, 1)
Xc_va, yc_va = load_dir(validation_cats_dir, 0)
Xd_va, yd_va = load_dir(validation_dogs_dir, 1)

X = np.array(Xc_tr + Xd_tr + Xc_va + Xd_va, dtype=np.uint8)
y = np.array(yc_tr + yd_tr + yc_va + yd_va, dtype=np.int32)


datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="reflect"
)

augmented, labels_aug = [], []
generated = 0
for xb, yb in datagen.flow(X, y, batch_size=64, shuffle=True):
    augmented.append(xb)
    labels_aug.append(yb)
    generated += len(xb)
    if generated >= 1000:
        break

X_aug = np.concatenate(augmented)[:1000]
y_aug = np.concatenate(labels_aug)[:1000]

X_all = np.concatenate([X, X_aug], axis=0)
y_all = np.concatenate([y, y_aug], axis=0)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, test_size=0.30, random_state=42, stratify=y_all
)

print("y_train.shape:", y_train.shape)


y_train.shape: (2800,)


In [11]:
# VGG19
import tensorflow as tf
from tensorflow.keras.applications import VGG19

def count_trainable_params(layer):
    return int(sum(tf.size(w).numpy() for w in layer.trainable_weights))

model = VGG19(include_top=False, weights=None, input_shape=(128, 220, 3))

conv_layers = [l for l in model.layers if "conv" in l.name and hasattr(l, "kernel")]

layer5 = conv_layers[4]
print("Camada:", layer5.name)

params = count_trainable_params(layer5)
print("Parâmetros treináveis (5ª conv da VGG19):", f"{params:,}".replace(",", "."))


Camada: block3_conv1
Parâmetros treináveis (5ª conv da VGG19): 295.168


In [12]:
# VGG19
import tensorflow as tf
from tensorflow.keras.applications import VGG19

model = VGG19(include_top=False, weights=None, input_shape=(128, 220, 3))

conv_layers = [l for l in model.layers if "conv" in l.name and hasattr(l, "kernel")]

layer5 = conv_layers[4]

print("Camada:", layer5.name)
print("Saída (TensorShape):", layer5.output.shape)

out_shape = layer5.output.shape
print("H×W×C:", int(out_shape[1]), "×", int(out_shape[2]), "×", int(out_shape[3]))


Camada: block3_conv1
Saída (TensorShape): (None, 32, 55, 256)
H×W×C: 32 × 55 × 256


In [13]:
# Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range=[0.8, 1.2],
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="reflect"
)

print("width_shift_range =", datagen.width_shift_range)


width_shift_range = 0.2
